In [42]:
from sage.categories.hopf_algebras import HopfAlgebras
from sage.combinat.posets.posets import FinitePoset
from sage.combinat.permutation import Permutations

In [43]:
class DoublePoset():
    def __init__(self, rel1, rel2, elements=None):
        self._P1 = Poset((elements, rel1))
        self._P2 = Poset((elements, rel2))

        # check: same ground set
        if set(self._P1) != set(self._P2):
            raise ValueError("The two posets must have the same ground set")

    def __len__(self):
        return self._P1.cardinality()

    def elements(self):
        return set(self._P1)

    def poset(self, i):
        if i == 2:
            return self._P1
        else:
            return self._P2

    def leq(self, i, a, b):
        return self.poset(i).is_lequal(a, b)
    
    def compose(self, other):
        """
        Return composition of two double posets

        EXAMPLES::

            
        TESTS::
        
        """
        E1 = self.poset(1)
        F1 = other.poset(1)
        E2 = self.poset(2)
        F2 = other.poset(2)

        #check: disjoint ground sets
        if set(E1) != set(F1):
            raise ValueError("Double posets must be disjoint for doing compsition")

        # First order
        rel1 = list(E1.cover_relations()) + list(F1.cover_relations())

        # Second order
        rel2 = list(E2.cover_relations()) + list(F2.cover_relations())
        for e in E2:
            for f in F2:
                rel2.append((e, f))

        elements = list(E1) + list(F1)
        return DoublePoset(rel1, rel2, elements=elements)

    def decompositions(self):
        """
        Return all decompositions of this double poset as pairs of double posets
        """
        decomps = []
        P1 = self.poset(1)
        P2 = self.poset(2)
        ground_set = set(self.elements())

        for G in Subsets(ground_set):  
            I = set(P1.order_ideal(G))
            S = ground_set - I

            # check if decomposition already done
            if any(I == set(d[0].elements()) for d in decomps):
                continue

            # Induce subposets
            P1_I = P1.subposet(I)
            P2_I = P2.subposet(I)
            P1_S = P1.subposet(S)
            P2_S = P2.subposet(S)

            D_I = DoublePoset(P1_I.cover_relations(), P2_I.cover_relations(), elements=list(I))
            D_S = DoublePoset(P1_S.cover_relations(), P2_S.cover_relations(), elements=list(S))
            decomps.append((D_I, D_S))

        return decomps


In [44]:
def number_of_pictures(D1, D2):
    """
    Count the number of pictures between two double posets D1 and D2.
    """
    E = list(D1.elements())
    F = list(D2.elements())

    if len(E) != len(F):
        return 0

    n = len(E)
    count = 0

    # get all bijections
    for sigma in Permutations(n):
        phi = {E[i]: F[sigma[i]-1] for i in range(n)}
        phi_inv = {v: k for k, v in phi.items()}

        # Forward: e <_1 e′ => phi(e) <_2 phi(e′)
        forward = True
        for e1 in E:
            for e2 in E:
                if e1 != e2 and D1.leq(1, e1, e2):
                    if not D2.leq(2, phi[e1], phi[e2]):
                        forward = False
                        break
            if not forward:
                break
        if not forward:
            continue

        # Backward: f <_1 f′ => phi_invs(f) <_2 phi_invs(f′)
        backward = True
        for f1 in F:
            for f2 in F:
                if f1 != f2 and D2.leq(1, f1, f2):
                    if not D1.leq(2, phi_inv[f1], phi_inv[f2]):
                        backward = False
                        break
            if not backward:
                break

        if backward:
            count += 1

    return count


In [45]:
E = DoublePoset([(1,2)], [(1,2)], elements=[1,2])
F = DoublePoset([(3,4)], [(3,4)], elements=[3,4])
number_of_pictures(E, F)
# !!!!!!!!!!!!!!!!!! haven't figured out the bug

1

In [ ]:
def internal_product_helper(D1, D2):
    r"""
    Return all D1 x_phi D2 for all increasing bijections phi 
    (the internal product = the sum of these)
    """
    
    E = D1.elements()
    F = D2.elements()

    if len(E) != len(F):
        return []

    n = len(E)
    result = []

    for sigma in Permutations(n):
        phi = {E[i]: F[sigma[i]-1] for i in range(n)}

        # Check increasing
        is_increasing = True
        for e1 in E:
            for e2 in E:
                if e1 != e2 and D1.leq(1, e1, e2):
                    if not D2.leq(2, phi[e1], phi[e2]):
                        is_increasing = False
                        break
            if not is_increasing:
                break

        if not is_increasing:
            continue

        # the graph under the increasing bijection
        elements = [(e, phi[e]) for e in E]

        # first order: compare (e, f) <_1 (e', f') iff f <_1 f' in D2
        cov1 = []
#         !!!!!!! Incomplete here !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        # second order
        cov2 = []

        D_phi = DoublePoset(cov1, cov2, elements=elements)
        result.append(D_phi)

    return result
